This code give the Monte Carlo fit and error propogation for Us-up and T-Us on the principal Hugoniot of olivine. This document is for submission with Chidester et al. (2021) at GRL.

Read in data

In [1]:
import numpy as np

##All olivine liquid data for fitting Us-up relation
upol = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=1,usecols=[6],max_rows=27)
upolerr = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=1,usecols=[7],max_rows=27)
Usol = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=1,usecols=[4],max_rows=27)
Usolerr = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=1,usecols=[5],max_rows=27)

ZUs_temp = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=28,usecols=[4],max_rows=5)
ZUs_temp_err = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=28,usecols=[5],max_rows=5)
ZT_temp = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=28,usecols=[12],max_rows=5)
ZT_temp_err = np.genfromtxt('SupplementalTable_1.csv',delimiter=',',skip_header=28,usecols=[13],max_rows=5)

# decaying shock data
#Us,T,reflectivity from Omega
s30744 = np.transpose(np.loadtxt('Olivine_30744TvsUs_4.txt',skiprows=0))
s30747 = np.transpose(np.loadtxt('Olivine_30747TvsUs_4.txt',skiprows=0))
s30750 = np.transpose(np.loadtxt('Olivine_30750TvsUs_5.txt',skiprows=0))
s30753 = np.transpose(np.loadtxt('Olivine_30753TvsUs_4.txt',skiprows=0))
s31844 = np.transpose(np.loadtxt('Olivine_31844TvsUs_4.txt',skiprows=0))
s31849 = np.transpose(np.loadtxt('Olivine_31849TvsUs_4.txt',skiprows=0))
s31854 = np.transpose(np.loadtxt('Olivine_31854TvsUs_4.txt',skiprows=0))
s31859 = np.transpose(np.loadtxt('Olivine_31859TvsUs.txt',skiprows=0))


Monte Carlo fit to Us-up and T-Us data

In [2]:
import matplotlib as mpl
import numpy as np
import pandas as pd
import scipy as sp
import pylab as py
from scipy.optimize import curve_fit, least_squares
from matplotlib import rc
import sys
import json
import numpy.random as random

#Us-up relation functions
def quad_hug(up,a,b,c):
    return a + b * up + c * up**2
def quad_hug_fit(p,up,Us):
    a = p[0]
    b = p[1]
    c = p[2]
    return a + b * up + c * up**2 - Us

#Monte Carlo Olivine Us-up fit
steps=10000
p=np.zeros((3,steps))
for i in range(steps):
    #Fitting to raw data + perturbations
    Ustest = Usol + (Usolerr * sp.randn())
    uptest = upol + (upolerr * sp.randn())

    temp1 = least_squares(quad_hug_fit,(1,1,1),args = (uptest,Ustest)) # Data Us-up fit
    p[:,i] = temp1.x
        
afit = np.mean(p[0])
afiter = np.std(p[0])
bfit = np.mean(p[1])
bfiter = np.std(p[1])
cfit=np.mean(p[2])
cfiter = np.std(p[2])
params = np.cov(p)

print('Us-up fit')
print('a, aerr',afit,afiter)
print('b, berr',bfit,bfiter)
print('c, cerr',cfit,cfiter)
print('covar',params)
lmat=sp.linalg.cholesky(params,lower=True)


#Temperature-Us function
def Tfit(Us,A,B,C):
    return A + B * Us + C * Us**2

#T-Us fit
steps=10000
A=np.zeros(steps)
B=np.zeros(steps)
C=np.zeros(steps)
D=np.zeros(steps)
#assuming 1% error for Us and 5% error on Temperature
for i in range(steps):
    #Fitting to raw data + perturbations
    Ucomp=[]

    rando=sp.randn()

    Ucomp.extend(s30744[0] + (0.01*s30744[0] * sp.randn()))

    Ucomp.extend(s30747[0] + (0.01*s30747[0] * sp.randn()))

    Ucomp.extend(s30750[0] + (0.01*s30750[0] * sp.randn()))

    Ucomp.extend(s30753[0] + (0.01*s30753[0] * sp.randn()))
    
    Ucomp.extend(s31844[0] + (0.01*s31844[0] * sp.randn()))
    
    Ucomp.extend(s31849[0] + (0.01*s31849[0] * sp.randn()))
    
    Ucomp.extend(s31854[0] + (0.01*s31854[0] * sp.randn()))
    
    Ucomp.extend(s31859[0] + (0.01*s31859[0] * sp.randn()))
    
    Ucomp.extend(ZUs_temp + ZUs_temp_err * sp.randn())

    Tcomp=[]

    Tcomp.extend(s30744[3] + (0.05*s30744[3] * sp.randn()))

    Tcomp.extend(s30747[3] + (0.05*s30747[3] * sp.randn()))

    Tcomp.extend(s30750[3] + (0.05*s30750[3] * sp.randn()))

    Tcomp.extend(s30753[3] + (0.05*s30753[3] * sp.randn()))
    
    Tcomp.extend(s31844[3] + (0.05*s31844[3] * sp.randn()))
    
    Tcomp.extend(s31849[3] + (0.05*s31849[3] * sp.randn()))
    
    Tcomp.extend(s31854[3] + (0.05*s31854[3] * sp.randn()))
    
    Tcomp.extend(s31859[3] + (0.05*s31859[3] * sp.randn()))
    
    Tcomp.extend(ZT_temp + ZT_temp_err * sp.randn())


    ########No monte carlo Error Fit##### Just Covariance
    UT=Ucomp
    TT=Tcomp
                  
    temp1, temp2 = curve_fit(Tfit, UT, TT,absolute_sigma=True) # Data T fit
    
    A[i]=temp1[0]
    B[i]=temp1[1]
    C[i]=temp1[2]

#P=sp.where(B<100000)
A_mean=np.mean(A)
A_std=np.std(A)
B_mean=np.mean(B)
B_std=np.std(B)
C_mean=np.mean(C)
C_std=np.std(C)

#Calculate covariance
X=[]
X.append(A)
X.append(B)
X.append(C)
CovarT=np.cov(X)
#Covar=stats.cov(X)

print('T-Us fit')
print("A, Aerr",A_mean,A_std)
print("B, Berr",B_mean,B_std)
print("C, Cerr",C_mean,C_std)
print("Tcovar",CovarT)

Us-up fit
a, aerr 3.9349418011693045 0.1091209811946605
b, berr 1.6378440627837276 0.03766289415564097
c, cerr -0.012676566605452366 0.0019467180054171254
covar [[ 1.19085794e-02 -2.38047888e-03  1.74269170e-04]
 [-2.38047888e-03  1.41863546e-03 -6.89668998e-05]
 [ 1.74269170e-04 -6.89668998e-05  3.79009000e-06]]
T-Us fit
A, Aerr 22093.067517470914 2841.3025792883054
B, Berr -3991.0225732797817 410.88874415647655
C, Cerr 209.58295195373125 14.81463256058699
Tcovar [[ 8.07380773e+06 -1.15778336e+06  4.04104905e+04]
 [-1.15778336e+06  1.68846445e+05 -6.01082636e+03]
 [ 4.04104905e+04 -6.01082636e+03  2.19495287e+02]]


Reflectivity - Us fit

In [3]:
from matplotlib.ticker import ScalarFormatter
import matplotlib as mpl
import numpy as np
import scipy as sp
import matplotlib.cm as cm
from scipy.optimize import least_squares
from scipy import odr

#Reflectivity data with 5% error
RT = np.genfromtxt('RvUs.csv',delimiter=',',usecols=[0])
RTe = RT*0.05
#Us data with 1% error
UT = np.genfromtxt('RvUs.csv',delimiter=',',usecols=[1])
UTe = UT*0.01

##Fit to reflectivity vs. Us from Igor
def HillPower(p,Us):
    return p[0] + ((p[1] - p[0]) * Us**p[2] / (p[3]**p[2] + Us**p[2]))

we = 1/(UTe**2)
wd = 1/(RTe**2)

mod = sp.odr.Model(HillPower)
data = sp.odr.RealData(UT,RT, UTe,RTe)
myodr = sp.odr.ODR(data, mod, beta0=[0.1, 25,11.,19.])
myoutput = myodr.run()
myoutput.pprint()

Beta: [ 0.08181378 31.03903275 11.27009468 19.73818428]
Beta Std Error: [0.00196042 0.5220217  0.05853726 0.05395045]
Beta Covariance: [[ 6.20252856e-07 -6.34795998e-05  1.41691088e-05 -8.78600258e-06]
 [-6.34795998e-05  4.39792426e-02 -3.24108427e-03  4.23508328e-03]
 [ 1.41691088e-05 -3.24108427e-03  5.53013078e-04 -4.35558199e-04]
 [-8.78600258e-06  4.23508328e-03 -4.35558199e-04  4.69743537e-04]]
Residual Variance: 6.196256235370569
Inverse Condition #: 0.013955718115930887
Reason(s) for Halting:
  Sum of squares convergence
